In [114]:
#import libraries
import pandas as pd
import numpy as np
from statistics import mean 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

#import LDA and PCA data
from PCAandLDA import PCA_batStats, batStats1D_LDA1, batStats2D_PCA

#add 1D array to bat_stats dataframe
data = batStats1D_LDA1.astype(float)

data2D = batStats2D_PCA

#USER PRARAMETERS DEFINED
#m = max number of clusters set by user
m=3
#a = alpha, threshold of dissimilarity set by user
a=0.05

#define distance measurment
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

#load data set
PCA_2D = pd.DataFrame(data2D, columns = ['PCA1', 'PCA2']).sample(2000).reset_index()
PCA_2D = PCA_2D[['PCA1', 'PCA2']]

#t = counter, counts number of iterations or clusters created
t=1

#initialize cluster dictionary
clusters_PCA = dict(zip(range(m), [[] for i in range(m)]))

#assign the first data point to cluster 1
clusters_PCA[0].append(PCA_2D.loc[0])

In [115]:
PCA_2D.head()
#np.mean(PCA_2D.loc[0:10])

,PCA1,PCA2
0,-0.549417,-0.473699
1,-0.799111,-0.787889
2,-0.509735,-0.323957
3,-0.850076,-0.614195
4,-0.797941,-0.284866


In [116]:
row = 1
while row < PCA_2D.shape[0]:
#add at least 1 data point to each cluster   
#if counter is less than max clusters aka not all clusters have been instantiated
    c=0 
    currentclusters=t
    distance = dict(zip(range(currentclusters), [[] for i in range(currentclusters)]))
    
    for c in range(currentclusters):
        #find distance for each cluster using euclidean distance
        distance[c].append(np.linalg.norm(PCA_2D.loc[row]-(np.mean(clusters_PCA[c]))))
    
    shortest_dist=min(distance.values())
    closest_cluster=list(distance.keys())[list(distance.values()).index(shortest_dist)]
    
    if t<m and shortest_dist[0] >a:
        print('cluster ' + str(c+1) + ' is made')
        clusters_PCA[currentclusters].append(PCA_2D.loc[row])
        t=t+1
    #after all clusters have at least one data point sort the rest of the data point by choosing cluster with closest centroid
    else:  
        clusters_PCA[closest_cluster].append(PCA_2D.loc[row])

    row = row + 1

#print cluster array
#clusters_PCA


cluster 1 is made
cluster 2 is made


In [117]:
c_df2=pd.DataFrame()
for c in clusters_PCA:
    for i in clusters_PCA[c]:
        c_df2=c_df2.append({'PCA1': i[0], 'PCA2': i[1], 'Cluster': c}, ignore_index=True)

c_df2.head()

,PCA1,PCA2,Cluster
0,-0.549417,-0.473699,0.0
1,-0.797941,-0.284866,0.0
2,-0.511721,-0.476288,0.0
3,-0.352948,-0.784853,0.0
4,-0.575107,-0.541077,0.0


In [118]:
#Create 2D plot with clusters
#create 1D Plot with clusters 
fig2=go.Figure()

fig2=px.scatter(x=c_df2.PCA1, y=c_df2.PCA2, color=c_df2.Cluster, 
               title="BSCA with 2D PCA BatStats Data:  alpha=" + str(a) + ", max clusters=" + str(m))

fig2.update_layout(coloraxis_colorbar=dict(
    title="Clusters",
    tickvals=[0,1,2,3, 4],
    ticktext=["c1","c2","c3","c4", "c5"],
    lenmode="pixels", len=500,
))

#fig.add_trace(go.scatter(x=clusters[1], y=range(len(clusters[0]))))
fig2.write_html('result2.html', auto_open=True)